# Tahap 2: Preprocessing Teks Berita Transjakarta

## Pastikan Anda sudah menginstal library yang dibutuhkan:
## pip install pandas sastrawi nltk

In [5]:
import pandas as pd
import re
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [6]:
# --- Setup Awal (Download kamus NLTK jika belum ada) ---
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt')

In [7]:
# --- 1. Membaca File Hasil Scraping ---
input_filepath = '../Data/hasil_scraping_transjakarta.csv'
try:
    df = pd.read_csv(input_filepath, sep=';')
    print(f"File '{input_filepath}' berhasil dibaca. Jumlah data awal: {len(df)} baris.")
except FileNotFoundError:
    print(f"Error: File '{input_filepath}' tidak ditemukan. Pastikan file ini ada di folder yang sama.")
    exit()

File '../Data/hasil_scraping_transjakarta.csv' berhasil dibaca. Jumlah data awal: 150 baris.


In [8]:
# --- 2. Pembersihan Awal ---
# Buang baris yang tidak punya isi berita
df.dropna(subset=['Isi Berita'], inplace=True)
df = df[~df['Isi Berita'].str.contains("Konten tidak ditemukan", na=False)]
print(f"Jumlah data setelah membuang konten kosong: {len(df)} baris.")

# Gabungkan Judul dan Isi Berita menjadi satu kolom
df['Teks_Lengkap'] = df['Judul'].astype(str) + " " + df['Isi Berita'].astype(str)

Jumlah data setelah membuang konten kosong: 150 baris.


In [9]:
# --- 3. Fungsi-fungsi untuk Preprocessing ---

# a. Case Folding
def case_folding(text):
    return text.lower()

# b. Pembersihan Noise
def clean_text(text):
    text = re.sub(r'https?://\S+|www\.\S+', '', text) # Hapus URL
    text = re.sub(r'<.*?>', '', text) # Hapus tag HTML
    text = re.sub(r'[^a-zA-Z\s]', '', text) # Hapus angka dan tanda baca
    text = re.sub(r'\s+', ' ', text).strip() # Hapus spasi berlebih
    return text

# c. Normalisasi Kata (contoh kamus, bisa ditambahkan)
def normalize_words(text):
    slang_dict = {
        'yg': 'yang', 'ga': 'tidak', 'gak': 'tidak', 'utk': 'untuk', 'dgn': 'dengan',
        'kalo': 'kalau', 'aja': 'saja', 'udah': 'sudah', 'transj': 'transjakarta',
        'mrt': 'moda raya terpadu', 'lrt': 'lintas rel terpadu'
        # Tambahkan kata lainnya jika ditemukan
    }
    words = text.split()
    normalized_words = [slang_dict.get(word, word) for word in words]
    return ' '.join(normalized_words)

# d. Stopword Removal
stopword_factory = StopWordRemoverFactory()
stopword_list = stopword_factory.get_stop_words()
# Tambahkan stopword custom jika perlu
stopword_list.extend(['rp', 'dki', 'jakarta']) 
stopword_remover = stopword_factory.create_stop_word_remover()

def remove_stopwords(text):
    return stopword_remover.remove(text)

# e. Stemming
stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()

def stem_text(text):
    return stemmer.stem(text)

In [10]:
# --- 4. Menjalankan Pipeline Preprocessing ---
print("\nMemulai proses preprocessing...")

# Terapkan semua fungsi ke kolom 'Teks_Lengkap'
# Proses ini mungkin memakan waktu beberapa menit, terutama stemming
df['Teks_Bersih'] = df['Teks_Lengkap'].apply(case_folding)
df['Teks_Bersih'] = df['Teks_Bersih'].apply(clean_text)
df['Teks_Bersih'] = df['Teks_Bersih'].apply(normalize_words)
df['Teks_Bersih'] = df['Teks_Bersih'].apply(remove_stopwords)
print("Stopword removal selesai.")
df['Teks_Stemmed'] = df['Teks_Bersih'].apply(stem_text)
print("Stemming selesai.")

print("Preprocessing selesai. Jumlah data akhir:", len(df))


Memulai proses preprocessing...
Stopword removal selesai.
Stemming selesai.
Preprocessing selesai. Jumlah data akhir: 150


In [11]:
# --- 5. Menyimpan Hasil ---
output_filepath = '../Data/hasil_preprocessing_transjakarta.csv'
# Pilih kolom yang relevan untuk disimpan
kolom_untuk_disimpan = [
    'No', 'Link', 'Sumber', 'Tanggal', 'Tag', 'PIC',
    'Teks_Lengkap', 'Teks_Bersih', 'Teks_Stemmed'
]
df_final = df[kolom_untuk_disimpan]
df_final.to_csv(output_filepath, index=False, encoding='utf-8')

print(f"\n✅ Proses preprocessing selesai. Hasil disimpan di file: {output_filepath}")

# Menampilkan 5 baris pertama dari hasil akhir
print("\nCuplikan 5 baris pertama dari data hasil preprocessing:")
print(df_final[['Teks_Lengkap', 'Teks_Stemmed']].head())


✅ Proses preprocessing selesai. Hasil disimpan di file: ../Data/hasil_preprocessing_transjakarta.csv

Cuplikan 5 baris pertama dari data hasil preprocessing:
                                        Teks_Lengkap  \
0  Hore! Naik Transjakarta, MRT, LRT Jakarta Rp 1...   
1  Sejumlah Layanan Transjakarta Terganggu Imbas ...   
2  Truk Mogok di Jalan Gatsu Arah Cawang, Sejumla...   
3  Rano Karno: Rute TransJakarta Ancol-Blok M Aka...   
4  Bus Listrik Transjakarta Kecelakaan di Setiabu...   

                                        Teks_Stemmed  
0  hore transjakarta moda raya padu lintas rel pa...  
1  layan transjakarta ganggu imbas demo daftar ru...  
2  truk mogok jalan gatsu arah cawang rute transj...  
3  rano karno rute transjakarta ancolblok luncur ...  
4  bus listrik transjakarta celaka setiabudi rem ...  
